In [55]:
import numpy as np
import pandas as pd

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [59]:
from xgboost import XGBClassifier

In [61]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

In [63]:
from aif360.algorithms.preprocessing import (
    Reweighing,
    DisparateImpactRemover,
    LFR
)

In [65]:
from aif360.algorithms.inprocessing import (
    MetaFairClassifier,
    GerryFairClassifier,
    PrejudiceRemover,
    ExponentiatedGradientReduction,
    GridSearchReduction,
    ARTClassifier,
    AdversarialDebiasing
)

In [67]:
from aif360.algorithms.postprocessing import (
    RejectOptionClassification,
    CalibratedEqOddsPostprocessing,
    EqOddsPostprocessing
)

In [69]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [211]:
def load_gmsc():
    df = pd.read_csv(r"C:\Users\niyat\OneDrive\Documents\Course Content\CS516\Project\CS516_Project\gmsc-training.csv")
    df = df.drop(columns=["Unnamed: 0"])  
    df.rename(columns={"SeriousDlqin2yrs": "label"}, inplace=True)

    # Treat age <= 25 as unprivileged
    df['age_binary'] = df['age'].apply(lambda x: 1 if x > 25 else 0)

    # Fill missing values if any (basic cleaning)
    df.fillna(df.median(), inplace=True)

    print("Null counts after filling:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

    X = df.drop(columns=['label', 'age_binary', 'age'])
    y = df['label']
    protected = df['age_binary']

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
    )

    return X_train, X_test, y_train, y_test, prot_train, prot_test

In [213]:
load_gmsc()

Null counts after filling:
Series([], dtype: int64)


(array([[-0.02421753, -0.10041896, -0.17316509, ..., -0.90128301,
         -0.05785249, -0.66612604],
        [-0.02408498, -0.10041896, -0.17301992, ..., -0.01614492,
         -0.05785249,  0.23720186],
        [-0.02418571, -0.10041896, -0.17289105, ..., -0.01614492,
         -0.05785249, -0.66612604],
        ...,
        [-0.02421753, -0.10041896, -0.17306934, ..., -0.01614492,
         -0.05785249,  0.23720186],
        [-0.02051314, -0.10041896, -0.17303615, ..., -0.90128301,
         -0.05785249, -0.66612604],
        [-0.0202136 ,  0.138087  ,  0.25173813, ..., -0.90128301,
          0.18281181, -0.66612604]]),
 array([[-0.02421753, -0.10041896, -0.17318459, ..., -0.90128301,
          0.18281181, -0.66612604],
        [-0.02419443, -0.10041896, -0.17319631, ..., -0.01614492,
         -0.05785249, -0.66612604],
        [-0.02098353, -0.10041896, -0.1729547 , ...,  0.86899317,
         -0.05785249,  0.23720186],
        ...,
        [-0.0202136 , -0.10041896, -0.17319827, ..., -

In [233]:
def load_give_me_some_credit():
    df = pd.read_csv(r"C:\Users\niyat\OneDrive\Documents\Course Content\CS516\Project\CS516_Project\gmsc-training.csv")
    df.rename(columns={"SeriousDlqin2yrs": "label"}, inplace=True)
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)
    df.fillna(df.median(), inplace=True)

    X_raw = df.drop(columns=['label', 'age', 'age_binary'])
    y = df['label'].values
    prot = df['age_binary'].values

    return X_raw, y, prot

def preprocess_data(X_raw, y, prot):
    cat_cols = X_raw.select_dtypes(include='object').columns.tolist()
    num_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    X_raw_train, X_raw_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_raw_train[cat_cols])
    X_test_cat = encoder.transform(X_raw_test[cat_cols])

    X_train_num = X_raw_train[num_cols].values
    X_test_num = X_raw_test[num_cols].values

    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test

In [241]:
def load_gmsc_inprocess():
    df = pd.read_csv(r"C:\Users\niyat\OneDrive\Documents\Course Content\CS516\Project\CS516_Project\gmsc-training.csv")
    df.rename(columns={"SeriousDlqin2yrs": "label"}, inplace=True)
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)
    df.fillna(df.median(), inplace=True)

    X_raw = df.drop(columns=['label', 'age', 'age_binary'])
    y = df['label'].values
    prot = df['age_binary'].values

    X_train_raw, X_test_raw, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()
    numeric_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_train_raw[categorical_cols])
    X_test_cat = encoder.transform(X_test_raw[categorical_cols])

    X_train_num = X_train_raw[numeric_cols].values
    X_test_num = X_test_raw[numeric_cols].values

    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test

In [75]:
def train_baseline_model(X_train, y_train, sample_weight=None):
    model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    return model

In [77]:
def evaluate_fairness(y_true, y_pred, prot, X=None):
    df = pd.DataFrame(np.hstack((X, y_true[:, None], prot[:, None])),
                      columns=[f"x{i}" for i in range(X.shape[1])] + ['label', 'protected'])

    dataset_true = BinaryLabelDataset(df=df,
                                      label_names=["label"],
                                      protected_attribute_names=["protected"],
                                      favorable_label=0, unfavorable_label=1)

    pred_dataset = dataset_true.copy()
    pred_dataset.labels = y_pred.reshape(-1, 1)

    metric = ClassificationMetric(dataset_true, pred_dataset,
                                  privileged_groups=[{'protected': 1}],
                                  unprivileged_groups=[{'protected': 0}])

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'disparate_impact': metric.disparate_impact(),
        'statistical_parity_difference': metric.statistical_parity_difference(),
        'equal_opportunity_difference': metric.equal_opportunity_difference()
    }

In [79]:
def evaluate_model(model, X_test, y_test, prot_test):
    y_pred = model.predict(X_test)
    return evaluate_fairness(y_test, y_pred, prot_test, X_test)

In [81]:
#XGboost
X_train, X_test, y_train, y_test, prot_train, prot_test = load_gmsc()
baseline_model = train_baseline_model(X_train, y_train)
baseline_metrics = evaluate_model(baseline_model, X_test, y_test.to_numpy(), prot_test.to_numpy())

baseline_metrics

Null counts after filling:
Series([], dtype: int64)


C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:55:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'accuracy': 0.9351111111111111,
 'disparate_impact': 0.9862524572485823,
 'statistical_parity_difference': -0.013419901443461502,
 'equal_opportunity_difference': -0.008154767458723122}

In [83]:
##disparate impact
def apply_disparate_impact_remover(X_train, y_train, prot_train, repair_level=1.0):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values if hasattr(y_train, 'values') else y_train
    df['protected'] = prot_train.values if hasattr(prot_train, 'values') else prot_train

    dataset = BinaryLabelDataset(
        favorable_label=0,  # <-- GOOD outcome = 0 for GMSC
        unfavorable_label=1,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    dir_remover = DisparateImpactRemover(repair_level=repair_level)
    repaired_dataset = dir_remover.fit_transform(dataset)

    X_repaired = pd.DataFrame(repaired_dataset.features)
    y_repaired = pd.Series(repaired_dataset.labels.ravel())
    prot_repaired = pd.Series(repaired_dataset.protected_attributes.ravel())

    return X_repaired, y_repaired, prot_repaired

In [87]:
inprocess_results = {}

In [89]:
# Apply DIR ONLY to training data
X_train_repaired, y_train_repaired, prot_train_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_train), y_train, prot_train
)

# Split repaired training data into sub-train and validation
X_subtrain, X_val, y_subtrain, y_val, prot_subtrain, prot_val = train_test_split(
    X_train_repaired, y_train_repaired, prot_train_repaired, test_size=0.3, random_state=42, stratify=y_train_repaired
)

# Train model on sub-train
fair_model = train_baseline_model(X_subtrain, y_subtrain)

# Evaluate model on repaired validation
fair_metrics = evaluate_model(
    fair_model,
    X_val.to_numpy(),
    y_val.to_numpy(),
    prot_val.to_numpy()
)
inprocess_results['Disparate_Impact_Remover'] = fair_metrics
fair_metrics

C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:59:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'accuracy': 0.9344444444444444,
 'disparate_impact': 0.9679337014750431,
 'statistical_parity_difference': -0.03142308135048755,
 'equal_opportunity_difference': -0.02367341050221161}

In [91]:
##LFR

In [111]:
def apply_lfr(X_train, y_train, prot_train):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    lfr = LFR(unprivileged_groups=[{'protected': 0}],
          privileged_groups=[{'protected': 1}],
          k=10, Ax=0.01, Ay=1.0, Az=0.1, verbose=0)

    lfr.fit(dataset)
    transformed_dataset = lfr.transform(dataset)

    X_transformed = pd.DataFrame(transformed_dataset.features)
    y_transformed = pd.Series(transformed_dataset.labels.ravel())
    prot_transformed = pd.Series(transformed_dataset.protected_attributes.ravel())

    return X_transformed, y_transformed, prot_transformed, lfr

In [113]:
X_train_lfr, y_train_lfr, prot_train_lfr, lfr_model = apply_lfr(pd.DataFrame(X_train), y_train, prot_train)

In [115]:
print("Unique labels in y_train_lfr:", np.unique(y_train_lfr))

Unique labels in y_train_lfr: [0. 1.]


In [117]:
fair_lfr_model = train_baseline_model(X_train_lfr, y_train_lfr)

C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:50:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [119]:
# Apply LFR transformation on test set
df_test = pd.DataFrame(X_test)
df_test['target'] = y_test.values
df_test['protected'] = prot_test.values

test_dataset = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=df_test,
    label_names=['target'],
    protected_attribute_names=['protected']
)

transformed_test_dataset = lfr_model.transform(test_dataset)

X_test_lfr = pd.DataFrame(transformed_test_dataset.features)
y_test_lfr = pd.Series(transformed_test_dataset.labels.ravel())
prot_test_lfr = pd.Series(transformed_test_dataset.protected_attributes.ravel())

# Evaluate
fair_lfr_metrics = evaluate_model(
    fair_lfr_model,
    X_test_lfr.to_numpy(),
    y_test_lfr.to_numpy(),
    prot_test_lfr.to_numpy()
)
inprocess_results['LFR']=fair_lfr_metrics

fair_lfr_metrics

{'accuracy': 0.9979111111111111,
 'disparate_impact': 0.998858484460229,
 'statistical_parity_difference': -0.0011414896791077656,
 'equal_opportunity_difference': -0.0011865043320410384}

In [121]:
print(np.unique(y_test_lfr, return_counts=True))

(array([0., 1.]), array([44908,    92], dtype=int64))


In [149]:
#reweighing

In [235]:
def apply_reweighing(X_train, y_train, prot_train):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=df_train,
        label_names=["label"],
        protected_attribute_names=["protected"]
    )
    RW = Reweighing(unprivileged_groups=[{'protected': 0}], privileged_groups=[{'protected': 1}])
    bld_rw = RW.fit_transform(bld_train)

    return bld_rw.features, bld_rw.labels.ravel(), bld_rw.instance_weights

In [237]:
X_train, X_test, y_train, y_test, prot_train, prot_test = preprocess_data(*load_give_me_some_credit())

X_rw, y_rw, sample_weights = apply_reweighing(X_train, y_train, prot_train)

expected_num_features = X_rw.shape[1]

if X_test.shape[1] < expected_num_features:
    padding = expected_num_features - X_test.shape[1]
    X_test_aligned = np.hstack((X_test, np.zeros((X_test.shape[0], padding))))
elif X_test.shape[1] > expected_num_features:
    X_test_aligned = X_test[:, :expected_num_features]
else:
    X_test_aligned = X_test

model_rw = train_baseline_model(X_rw, y_rw, sample_weight=sample_weights)

reweighing_metric = evaluate_model(model_rw, X_test_aligned, y_test, prot_test)

inprocess_results['reweighing'] = reweighing_metric
reweighing_metric

C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [03:40:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'accuracy': 0.9312666666666667,
 'disparate_impact': 0.9978325765982927,
 'statistical_parity_difference': -0.0020953775727402535,
 'equal_opportunity_difference': -0.0012583229239316873}

In [239]:
for model, metrics in inprocess_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()

🔹 Disparate_Impact_Remover
  accuracy: 0.9344
  disparate_impact: 0.9679
  statistical_parity_difference: -0.0314
  equal_opportunity_difference: -0.0237

🔹 LFR
  accuracy: 0.9979
  disparate_impact: 0.9989
  statistical_parity_difference: -0.0011
  equal_opportunity_difference: -0.0012

🔹 reweighing
  accuracy: 0.9313
  disparate_impact: 0.9978
  statistical_parity_difference: -0.0021
  equal_opportunity_difference: -0.0013



In [ ]:
#Inprocess

In [245]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_gmsc_inprocess()
results = {}

In [251]:
def train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train, label_names=["label"], protected_attribute_names=["protected"],
                                   favorable_label=1, unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test, label_names=["label"], protected_attribute_names=["protected"],
                                  favorable_label=0, unfavorable_label=1)

    clf = GerryFairClassifier(C=100, printflag=False, gamma=0.005, fairness_def='FP', max_iters=50)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [253]:
results['GerryFair'] = train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test)

In [255]:
def train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    clf = PrejudiceRemover(sensitive_attr="protected", eta=25.0)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [257]:
results['PrejudiceRemover'] = train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test)

In [259]:
def train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    expgrad = ExponentiatedGradientReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    expgrad.fit(bld_train)

    pred = expgrad.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [261]:
results['ExponentiatedGradient'] = train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test)

C:\Users\niyat\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [265]:
def train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    grid = GridSearchReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    grid.fit(bld_train)

    pred = grid.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [267]:
results['GridSearch'] = train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test)

C:\Users\niyat\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [269]:
tf.disable_eager_execution()

In [271]:
def train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test):
    # Train data
    tf.reset_default_graph()
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=0,
                                   unfavorable_label=1)

    # Test data
    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=0,
                                  unfavorable_label=1)

    # TensorFlow session
    sess = tf.Session()

    clf = AdversarialDebiasing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        scope_name='adv_debiasing',
        sess=sess,
        num_epochs=50,
        batch_size=64,
        debias=True
    )

    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [273]:
results['AdversarialDebiasing'] = train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

epoch 0; iter: 0; batch classifier loss: 0.608301; batch adversarial loss: 1.066470
epoch 0; iter: 200; batch classifier loss: 0.327045; batch adversarial loss: 0.928578
epoch 0; iter: 400; batch classifier loss: 0.300993; batch adversarial loss: 0.659402
epoch 0; iter: 600; batch classifier loss: 0.213995; batch adversarial loss: 0.452488
epoch 0; iter: 800; batch classifier loss: 0.268944; batch adversarial loss: 0.334165
epoch 0; iter: 1000; batch classifier loss: 0.155042; batch adversarial loss: 0.283616
epoch 0; iter: 1200; batch classifier loss: 0.225082; batch adversarial loss: 0.200638
epoch 0; iter: 1400; batch classifier loss: 0.135688; batch adversarial loss: 0.158124
epoch 0; iter: 1600; batch classifier loss: 0.317485; batch adversarial loss: 0.163876
epoch 1; iter: 0; batch classifier loss: 0.230486; batch adversarial loss: 0.169476
epoch 1; iter: 200; batc

In [275]:
print("Fairness Evaluation Across Compatible AIF360 In-processing Algorithms\n")
for model, metrics in results.items():
    print(f"🔹 {model}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()

Fairness Evaluation Across Compatible AIF360 In-processing Algorithms

🔹 GerryFair
  accuracy: 0.9332
  disparate_impact: 0.9575
  statistical_parity_difference: -0.0424
  equal_opportunity_difference: -0.0313

🔹 PrejudiceRemover
  accuracy: 0.0659
  disparate_impact: 0.3925
  statistical_parity_difference: -0.0027
  equal_opportunity_difference: -0.0019

🔹 ExponentiatedGradient
  accuracy: 0.9340
  disparate_impact: 0.9861
  statistical_parity_difference: -0.0139
  equal_opportunity_difference: -0.0105

🔹 GridSearch
  accuracy: 0.9338
  disparate_impact: 1.0035
  statistical_parity_difference: 0.0034
  equal_opportunity_difference: 0.0015

🔹 AdversarialDebiasing
  accuracy: 0.9358
  disparate_impact: 0.9561
  statistical_parity_difference: -0.0428
  equal_opportunity_difference: -0.0296



In [277]:
#postprocessing

In [279]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_gmsc_inprocess()
post_results = {}

In [303]:
def train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)
    bld_pred.labels = y_pred.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{'protected': 0}],
        privileged_groups=[{'protected': 1}],
        low_class_thresh=0.3, high_class_thresh=0.7,
        num_class_thresh=100, num_ROC_margin=50,
        metric_name="Statistical parity difference",
        metric_ub=0.05, metric_lb=-0.05
    )
    roc = roc.fit(bld_test, bld_pred)
    pred = roc.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [305]:
post_results['RejectOptionClassification'] = train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [05:04:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [285]:
def train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    bld_test = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)

    ceo = CalibratedEqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        cost_constraint="fnr",
        seed=42
    )
    ceo = ceo.fit(bld_test, bld_pred)
    pred = ceo.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [287]:
post_results['CalibratedEqOdds'] = train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [04:55:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [289]:
def train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=0,
        unfavorable_label=1,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.labels = y_pred.reshape(-1, 1)

    eq = EqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}]
    )
    eq = eq.fit(bld_test, bld_pred)
    pred = eq.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)

In [291]:
post_results['EqualizedOdds'] = train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

C:\Users\niyat\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [04:56:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [307]:
for model, metrics in post_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()

🔹 RejectOptionClassification
  accuracy: 0.0666
  disparate_impact: 219.8148
  statistical_parity_difference: 0.0492
  equal_opportunity_difference: 0.0261

🔹 CalibratedEqOdds
  accuracy: 0.0651
  disparate_impact: 1.7005
  statistical_parity_difference: 0.0175
  equal_opportunity_difference: 0.0117

🔹 EqualizedOdds
  accuracy: 0.9224
  disparate_impact: 0.9915
  statistical_parity_difference: -0.0082
  equal_opportunity_difference: 0.0001

